In [1]:
from elasticsearch import Elasticsearch
es = Elasticsearch('localhost:9200')
es.info()

/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'name': 'e4e1b4447e20',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'DNoCuE3RTv2p_fQWsmn1YA',
 'version': {'number': '7.15.1',
  'build_flavor': 'default',
  'build_type': 'deb',
  'build_hash': '83c34f456ae29d60e94d886e455e6a3409bba9ed',
  'build_date': '2021-10-07T21:56:19.031608185Z',
  'build_snapshot': False,
  'lucene_version': '8.9.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [2]:
path = '/opt/ml/data_v3/wikipedia_documents.json'

In [3]:
import json #json 수정하려고 열음
with open(path, 'r') as f:
    wiki = json.load(f)

In [4]:
wiki['224']

{'text': '당의 경선중 박근혜 측에서 BBK 의혹을 제기하면서부터 친이(親李, 친이명박) 진영과 친박(親朴, 친박근혜) 진영의 갈등이 시작되었다. 박근혜 측은 결과에 승복해 갈등은 사라지는 듯하다가, 본격적으로 갈등이 심화되어 집단탈당사태가 일어난건 공천파동에서였다. 이후 친박연대가 창당되면서 미디어법 등 몇 가지 정책들에서 친이계와 친박계의 미묘한 의견 차이는 있어왔으나, 2010년 1월 이명박 대통령이 세종시 사수안을 번복하고 제안한 세종시 수정안 방침에 대해 친박연대를 비롯한 당의 주요 친박계 의원들이 반발하면서 그 갈등이 더욱 커지고 있었다. 박근혜는 원안 추진을 재차 강조하여 화합의 가능성이 낮아진 데 이어 심지어 분당 가능성까지 나왔다 이전에 홍준표는 박근혜의 세종시 원안 추진론 발언에 대해 \'탈당\' 이야기까지 나온 상황이었고, 정몽준이 당론을 변경해야 한다는 말에 박근혜가 "국민의 신뢰를 잃은 것에 대해 책임지실 문제"라며 정몽준 책임론을 들고 나왔으며, 정운찬 국무총리를 비롯한 정부 고위직 관계자들까지 박근혜를 압박하는 상황이 벌어지면서 갈등의 골이 깊어지고 있었다. 이러한 갈등 속에서 야당들이 추진했던 정운찬 총리 해임안을 여당 내 친박계 의원들이 찬성한다는 입장을 밝히자 친이계 의원들은 "총리 해임안에 찬성하는 것은 분당하자는 것" 이라며 계파간의 대립이 극화되었다. 6월 29일 세종시와 관련해서 친박계 의원들을 비롯하여 야당의원들이 반대하는 가운데 수정안이 부결되었다. 이 때문에 친이, 친박간의 갈등이 당분간 해소되기 어렵다는 관측들이 나왔다. 하지만 2010년 8월, 이명박 대통령과 박근혜 전 대표가 전격 회동하면서 화해 분위기가 조성되었고 양측은 우호적인 분위기를 유지하다가 2011년에 들어서면서 친이계가 급속히 몰락하면서 당 중심이 친박계로 이동하였다는 관측이 지배적이다.',
 'title': '자유한국당'}

In [5]:
# for i in range(len(wiki)):
#     del wiki[str(i)]['corpus_source']# 필요없는거 삭제
#     del wiki[str(i)]['domain']
#     del wiki[str(i)]['author']
#     del wiki[str(i)]['html']
#     del wiki[str(i)]['document_id']
#     del wiki[str(i)]['url']

In [5]:
wiki['16']['text']

'통계학은 관찰 및 조사로 얻을 수 있는 데이터로부터, 응용 수학의 기법을 이용해 수치상의 성질, 규칙성 또는 불규칙성을 찾아낸다. 통계적 기법은, 실험 계획, 데이터의 요약이나 해석을 실시하는데 있어서의 근거를 제공하는 학문이며, 폭넓은 분야에서 응용되어 실생활에 적용되고 있다 통계학은 실증적인 뿌리를 가지고 있으며 실질적 활용에 초점을 맞추고 있기 때문에, 흔히 순수수학과는 다소 구분되는 응용수학의 일종으로 여겨진다. 통계학의 방법을 통해, 실제의 수치들을 왜곡하여 해석하는 것을 막고 연구를 바탕으로 합리적인 의사결정을 할 수 있다. 통계학은 과학, 산업, 또는 사회의 문제에 적용되며 모집단을 연구하는 과정이 우선시된다. 모집단은 "한나라 안에 사는 모든 사람" 또는 "크리스탈을 구성하는 모든 원자"와 같이 일정한 특성을 지닌 집단이면 어느 것이든 가능하다. 통계학자들은 전체인구(인구조사를 하는 기업)에 대한 데이터를 편집한다. 이것은 정부의 통계관련 법률요약집같은 조직화된 방법으로 수행될 수도 있다. 기술통계학은 모집단의 데이터를 요약하는데 사용된다. 도수 및 비율 (경주 등) 범주 형 데이터를 설명하는 측면에서 더 유용할 동안 수치 기술자는 연속적인 데이터 유형 (소득 등)에 대한 평균과 표준 편차를 포함한다. 데이터 분석 방법 엄청난 자료가 연구되는 현대 사회에서 경제지표연구, 마케팅, 여론조사, 농업, 생명과학, 의료의 임상연구 등 다양한 분야에서 응용되고 있는 통계는 단연 우리 사회에서 가장 필요하고 실용적인 학문이라고 할 수 있다.'

In [6]:
import re
for i in range(len(wiki)):    
    temp = wiki[f'{i}']['text']
    temp = "!!" + wiki[f'{i}']['title'] + "!!" + temp
    wiki[f'{i}']['text'] = temp

In [7]:
wiki['14']

{'text': '!!파이의 날!!파이의 날(Pi Day)은 원주율을 기념하는 날이다. 파이의 날은 원주율의 근삿값이 3.14이어서 3월 14일에 치러진다. 보통 3.14159에 맞추기 위해 오후 1시 59분에 기념하는데, 오후 1시 59분은 엄밀히 말하면 13시 59분이기 때문에 오전 1시 59분 혹은 15시 9분(오후 3시 9분)에 치러야 한다고 주장하는 사람도 있다. 세계 각국의 수학과에서 기념행사를 연다. 3월 14일은 알베르트 아인슈타인의 생일이면서 스티븐 호킹의 기일이기도 하다. 이 날은 여러 방법으로 기념된다. 사람들이 모여서 원주율이 생활에서 어떤 역할을 했는지 이야기하고 원주율이 없는 세상을 상상해 본다. 모임에서는 보통 파이를 먹는다. 또한 많은 행사에서 원주율을 소수점 아래의 숫자를 얼마나 많이 외우는지 겨루는 대회가 열린다. 분수 3과 7분의 1을 가분수로 나타내면 7분의 22가 되는데, 이를 유리수(순환소수)로 나타내면 3.142857...로 π의 근삿값이 되므로 7월 22일을 파이 근삿값 날로 부르기도 한다.',
 'title': '파이의 날'}

In [8]:
INDEX_NAME = 'wiki'
index_config = {
        "settings": {
            "analysis": {
                "filter":{
                    "my_stop_filter": { 
                        "type" : "stop",
                        "stopwords_path" : "my_stopwords.txt" # /etc/elastic안에 txt파일이 존재해야 댑니다
                    }
                },
                "analyzer": {
                    "nori_analyzer": {
                        "type": "custom",
                        "tokenizer": "nori_tokenizer", # 노리 형태소 깔아야대는데 에러나면 맨위에 참고해서 깔기
                        "decompound_mode": "none",
                        "filter" : ["my_stop_filter"]# 위에서 정의한 stopword
                    }
                }
            }
        },
        "mappings": {
            "dynamic": "strict", # 먼지 잘모르겟
            "properties": {
                "text": {"type": "text", "analyzer": "nori_analyzer"},
                "title":{"type": "text"}
                }
            }
        }

In [9]:
if es.indices.exists(index=INDEX_NAME):
    es.indices.delete(index=INDEX_NAME)
es.indices.create(index=INDEX_NAME, body=index_config)

/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
<ipython-input-9-c1b8263f83d6>:3: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index=INDEX_NAME, body=index_config)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'wiki'}

In [10]:
import time
import tqdm

for doc_id, doc in tqdm.tqdm(wiki.items(),total=len(wiki)):
    
    es.index(index=INDEX_NAME, id=doc_id, body=doc)
    # time.sleep(0.1) # 문서집어넣기 time슬립너어야 에러 안걸림 (너무빨리 문서를 호출해서)

  0%|          | 0/60613 [00:00<?, ?it/s]<ipython-input-10-5a9fb178b7ff>:6: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.index(index=INDEX_NAME, id=doc_id, body=doc)
100%|██████████| 60613/60613 [06:29<00:00, 155.47it/s]


In [11]:
import pprint
# doc = es.get(index=INDEX_NAME, id=60000)#
# pprint.pprint(doc)

In [12]:
tv = es.termvectors(index=INDEX_NAME, id=2, body={"fields" : ["content","title"]})

In [13]:
pprint.pprint(tv)

{'_id': '2',
 '_index': 'wiki',
 '_type': '_doc',
 '_version': 1,
 'found': True,
 'term_vectors': {'title': {'field_statistics': {'doc_count': 5768,
                                                 'sum_doc_freq': 9399,
                                                 'sum_ttf': 9404},
                            'terms': {'백남준': {'term_freq': 1,
                                              'tokens': [{'end_offset': 3,
                                                          'position': 0,
                                                          'start_offset': 0}]}}}},
 'took': 0}


In [14]:
from datasets import load_dataset

In [15]:
import datasets
import pandas as pd
import re
import pandas as pd
from datasets import load_from_disk
trainpath = "/opt/ml/data/train_dataset"
testpath = '/opt/ml/data/test_dataset'
datasets = load_from_disk(testpath)
datasets

DatasetDict({
    validation: Dataset({
        features: ['question', 'id'],
        num_rows: 600
    })
})

In [16]:
query = datasets['validation']['question']
# ground_truth = datasets['validation']['context']
id = datasets['validation']['id']
len(query)

600

In [17]:
id[0]

'mrc-1-000653'

In [18]:
correct = 0
total = []
for i, q in enumerate(query):
    q = q.replace('~','-')
    res = es.search(index=INDEX_NAME, q=q, size=10)
    # print('query : ', q)
    # print('id : ', id[i])
    # print(res)
    tmp = {
        'id':id[i],
    }
    tmp2 = {f'text{j+1}':res['hits']['hits'][j]['_source']['text'] for j in range(10)}
    tmp3 = {f'score{j+1}':res['hits']['hits'][j]['_score'] for j in range(10)}
    tmp.update(tmp2)
    tmp.update(tmp3)
    
    total.append(tmp)
    
    # context = res['hits']['hits']
    # print(q)
    # print(context)
    # print('='*100)


# print('% : ', correct/len(query))

/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [23]:
query[0]

"유령'은 어느 행성에서 지구로 왔는가?"

In [22]:
total[0]['text1']

'!!더크 젠틀리의 성스러운 탐정 사무소!!더크 젠틀리의 성스러운 탐정사무소의 줄거리는 이야기의 중추적인 부위에 자리잡은 시간 여행이란 주제 때문에 줄거리가 이어져 있지 않다. 40억년전 지구에서, 사락사라의 사람들이 자신들만의 낙원을 만들기 위해 지구에 착륙하였다. 그러나, 기술자의 게으름으로 인해서 착륙선은 폭발하였으며, 안에 타고 있던 사락사라 사람들은 모두다 죽어버렸으며 그 폭발에 의한 에너지가 아미노산을 만들어 지구에 생명이 생기게 되었다. 그 게으른 엔지니어는 유령이 되어 그가 저지른 잘못을 돌리기 위해 돌아다니면서, 인류의 발전에 영향을 미쳐왔다. 1800년 초에 유령은 케임브리지 세인트 체드 단과대학의 리즈 교수가 타임머신을 가졌다는 걸 알게 되고, 리즈교수를 손에 넣을려고 하였으나 실패하고 만다. 그리고 유령은 새뮤얼 테일러 콜리지에게 붙어, 그의 시 쿠빌라이 칸과 늙은 뱃사람의 노래에 착륙선을 고칠방법을 적게 만든다. 유령은 200여 년 동안 콜리지의 작품에 감명을 받은 영혼을 찾아다녔으며, 21세기무렵 마이클 웬튼 윅스라는 전 예술잡지 편집장을 찾게 된다. 유령은 마이클이 콜리지의 작품을 읽게 만들었으며, 그를 조종하기 위해 마이클로 하여금 마이클의 후임 편집장인 알버트 로스를 살해하게 된다. 유령의 다음 행동은 다른 사람에게 영향을 끼칠 숙주를 찾는 일이었다. 리즈는 어린 소녀를 재미있게 해주기 위해 타임머신을 가동하였으며, 유령이 사용하기위해 데려온 전자 수도사는 웨이포인트 테크놀러지 II의 사장인 고든 웨이를 죽여버린다. 리즈의 학생이었으며, 웨이포인트 테크놀러지 II의 컴퓨터 프로그래머인 리처드 맥더프는 그의 여자친구인 수잔 웨이의 집의 자동응답기에 저장된 메시지를 지우기 위해서 몰래 집에 침입하게 되는데, 그때 맥더프는 알지 못하게 유령에게 이용되었다. 이건 만물의 상호 연관성을 믿는 맥더프의 친구인 더크 젠틀리를 끌여들이게 되었다. 더크 젠틀리와 맥더프는 여럿 우연한 장소에서 정보를 얻고, 유령이 개입했다는 것과, 리즈 교수가 타

In [19]:
import pandas as pd
df = pd.DataFrame(total)
df


,id,text1,text2,text3,text4,text5,text6,text7,text8,text9,...,score1,score2,score3,score4,score5,score6,score7,score8,score9,score10
0,mrc-1-000653,!!더크 젠틀리의 성스러운 탐정 사무소!!더크 젠틀리의 성스러운 탐정사무소의 줄거리...,!!지구 근접 소행성!!지구 근접 소행성(地球近接小行星)은 지구 궤도와 교차하는 궤...,!!지구 근접 소행성!!지구 근접 소행성(地球近接小行星)은 지구 궤도와 교차하는 궤...,!!글리제 581 g!!글리제 581 g는 천칭자리 방향으로 지구로부터 약 20.5...,!!서브지구!!서브지구(sub-Earth)는 지구와 금성보다 질량이 확연하게 작은 ...,!!페가수스자리 51 b!!페가수스자리 51 b가 발견되기 이전 폴란드 천문학자 알...,!!HD 40307 b!!HD 40307 b의 질량은 적어도 지구의 4.2배 수준인...,!!국제천문연맹의 행성의 정의!!이 정의는 엄밀해야 할 정의가 모호성을 띄고 있다는...,"!!HD 10180!!이 별은 여러 면에서 태양과 비슷하나, 질량은 태양보다 6퍼센...",...,17.102198,16.029920,16.029920,15.858599,15.712366,15.546904,15.496980,15.345918,15.276966,15.246025
1,mrc-1-001113,!!오퍼레이션7!!냉전 종식 이후 전 세계적으로 소규모의 끊임없는 국지 분쟁들이 생...,!!크리스 니코스키!!SK에서 방출된 이후 그는 일본을 여행하며 잠시 신변을 정리하...,!!코리 리오단!!2014년 1월 10일 KBO 리그 LG 트윈스와 계약하며 201...,!!옌볜 푸더!!1955년 8월 1일에 한인 위주로 구성된 <길림성축구팀>(지린축구...,!!에드미우송 디아스 지 루세나!!에드미우송은 1988년 20세의 나이로 포르투갈에...,!!브라이언 배스!!2012년 1월 18일 카림 가르시아의 대체 외국인 선수로 한화...,!!모르도비야 아레나!!2018년 FIFA월드컵 개최를 위해 제출된 팀 휘프의 새로...,!!내셔널리그 2010!!내셔널리그 2010(대한생명의 후원을 받아 대한생명 내셔널...,!!카를루스 아우베르투 파헤이라!!2002년 FIFA 월드컵이 끝나고 브라질의 감독...,...,17.460379,17.154230,15.657099,15.509413,15.164756,14.344352,14.001638,13.927957,13.894322,13.485586
2,mrc-0-002191,"!!엥겔베르트 돌푸스!!1933년 3월, 투표 과정의 위법성에 대한 문제제기가 불거...",!!엥겔베르트 돌푸스!!돌푸스는 니더외스터라이히주의 텍싱탈에서 싱글맘이자 신실한 가...,!!죠죠의 기묘한 모험 All Star Battle!!; 디오의 성 : 1부의 무대...,!!콘라트 3세 (부르군트)!!부르군트의 콘라트 3세(Conrad III/Konra...,"!!요숭!!650년 섬주(陝州) 협석(硤石)에서 요의(姚懿, 요선의姚善意)의 아들로...",!!이용수 (1928년)!!1928년 경상북도 성주에서 출생하여 지난날 한때 경상북...,!!금전신탁!!금전신탁의 운용방법은 특정한 특정(特定) 금전신탁과 특정하지 않은 불...,!!조지 S. 바우트웰!!바우트웰은 1869년 율리시스 S. 그랜트 대통령으로부터 ...,!!트라쉬불로스!!트라쉬불로스의 배경이나 초기 생애에 대해서는 알려진 것이 거의 없...,...,37.175800,20.547073,17.318540,16.901167,16.227207,16.097681,15.908007,14.557250,14.486744,14.122905
3,mrc-0-003951,!!서구화!!유럽인들의 아메리카와 오세아니아 식민지화로 인해 아메리카와 오세아니아의...,"!!멕시코의 언어!!농아인 공동체는 멕시코 수화와 유카탄 수화를 사용하고, 바하칼리...",!!멕시코의 언어!!멕시코의 언어는 다양하고 그 중 스페인어가 가장 널리 쓰인다. ...,"!!다중심언어!!영어는 발음, 어휘, 철자법 등이 영국, 북미, 영어를 사용하는 아...",!!국제 SIL!!국제 SIL은 에스놀로그에 기록되는 모든 언어에 3자리 부호를 매...,!!국제 SIL!!국제 SIL은 에스놀로그에 기록되는 모든 언어에 3자리 부호를 매...,!!세계농아인연맹!!세계농아인연맹(World Federation of the Dea...,!!니카라과 수화!!1970년대 이전에 니카라과에는 청각장애인의 커뮤니티가 없었다....,!!미국의 문화!!미국의 언어 미국은 연방 정부 차원에서 따로 공용어를 지정하지 않...,...,48.735043,34.802610,33.915123,29.701948,27.905252,27.905252,27.117690,26.584930,26.017113,25.392410
4,mrc-1-001272,!!보응우옌잡!!1926년 학생 시절 베트남청년혁명당에 가입했고 1930년에 학생 ...,!!깟비 국제공항!!제1차 인도차이나 전쟁 중에는 프랑스 공군의 깟비 공군기지로 이...,!!즈엉반민!!그는 1940년대에 자신의 군사 경력을 시작하여 프랑스 식민지 육군에...,!!명호작전!!이 작전에 관해서는 연합군의 침공 외의 전투는 일어나지 않았다. 일본...,!!러셀 법정!!러셀 법정은 1966년 영국의 철학자 버트런드 러셀이 베트남 전쟁에...,!!나산 공항!!공항이 폐쇄됨에 따라 현재 나산 공항에 대한 예약된 운항 서비스는 ...,!!베트남의 역사!!제1차 인도차이나 전쟁 이후 영국과 중국은 이전의 지배자였던 프...,!!베트남의 역사!!제1차 인도차이나 전쟁 이후 영국과 중국은 이전의 지배자였던 프...,"!!타이빈성!!과거 홍강 삼각주의 중심부에 자리 잡고 있었지만, 과거에는 타이빈성은...",...,31.681423,15.859224,14.538159,12.526538,12.057945,11.544473,11.247150,11.247150,10.815040,9.990664
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,mrc-0-002989,!!극장판 가면라이더 W FOREVER: A to Z/운명의 가이아 메모리!!재단 ...,!!극장판 가면라이더 W FOREVER: A to Z/운명의 가이아 메모리!!후토 ...,!!극장판 가면라이더 W FOREVER: A to Z/운명의 가이아 메모리!!NEV...,"!!극장판 가면라이더 W FOREVER: A to Z/운명의 가이아 메모리!!""가면...",!!IAS 머신!!IAS 머신은 40비트 워드를 갖춘 이진 방식의 컴퓨터의 하나로서...,!!ML (프로그래밍 언어)!!ML은 범용 프로그래밍 언어의 일종이다. 프로그래밍 ...,!!ML (프로그래밍 언어)!!ML은 범용 프로그래밍 언어의 일종이다. 프로그래밍 ...,"!!가이아 건담!!가이아 건담(Gaia Gundam, ガイアガンダム)은 TV 애니메...",!!FB-DIMM!!FB-DIMM(Fully Buffered DIMM)은 메모리 시...,...,29.985798,23.889470,22.279081,22.264990,19.711796,19.415108,19.415108,18.901388,18.155262,17.389938
596,mrc-0-001804,!!장면!!과거의 죄악에 대한 소급입법 적용을 반대하였다. 4.19 혁명 당시 사망...,!!장면!!과거의 죄악에 대한 소급입법 적용을 반대하였다. 4.19 혁명 당시 사망...,!!장면!!4.19 의거나 3.15 부정선거 당시 학생 시위 진압자에 대한 미온적인...,!!장면!!4.19 의거나 3.15 부정선거 당시 학생 시위 진압자에 대한 미온적인...,!!장면!!4.19 관련자에 대한 미온적인 처벌이 문제가 되자 그는 결국 1960년...,!!장면!!4.19 관련자에 대한 미온적인 처벌이 문제가 되자 그는 결국 1960년...,"!!장면!!1951년 장면이 주한 미국대사와 프랑스 

In [20]:
df.to_csv('../add_context_test_dataset.csv')